In [361]:
import pandas as pd

file_path = 'datos.xlsx'
df_weights = pd.read_excel(file_path, sheet_name='weights')
df_prices = pd.read_excel(file_path, sheet_name='Precios')

In [362]:
df_prices.reset_index(drop=False, inplace=True)
df_prices.rename(columns={'index': 'date_id'}, inplace=True)
df_prices[df_prices['Dates'] == "2022-02-15"]

,date_id,Dates,EEUU,Europa,Japón,EM Asia,Latam,High Yield,IG Corporate,EMHC,Latam HY,UK,Asia Desarrollada,EMEA,Otros RV,Tesoro,MBS+CMBS+AMBS,ABS,MM/Caja
0,0,2022-02-15,9383.57,66.03,390.26256,82.35,26.84,2355.25,3314.66,437.0424,1007.533,34.48,1212.81,290.449,717.81,2403.85,2226.56,368.44,104.64


In [363]:
initial_amount = 1_000_000_000
initial_date = "2022-02-15"
initial_prices = df_prices[df_prices['Dates'] == initial_date]

#obtain the columns of portafolios
portafolio_columns = [col for col in df_weights.columns if 'portafolio' in col]
portafolio_columns = {col: col.split(' ')[1] for col in portafolio_columns}

# Usar melt para consolidar las cantidades
df_weights = df_weights.melt(id_vars=['Fecha', 'activos'], 
                                value_vars=[f"{portafolio}" for portafolio in portafolio_columns.keys()], 
                                var_name='portafolio', 
                                value_name='weight')

# rename all the rows for each portafolio
df_weights['portafolio'] = df_weights['portafolio'].apply(lambda x: portafolio_columns[x])

df_weights.head()

,Fecha,activos,portafolio,weight
0,2022-02-15,EEUU,1,0.280
1,2022-02-15,Europa,1,0.087
2,2022-02-15,UK,1,0.023
3,2022-02-15,Japón,1,0.038
4,2022-02-15,Asia Desarrollada,1,0.016


In [364]:
#Calculate the cuantity of each asset
def calculate_actives_cuantity(row):
    activos = row['activos']
    price = initial_prices[activos]
    weight = row["weight"]
    return (weight * initial_amount)/ price

df_weights[f"cantidad"] = df_weights.apply(calculate_actives_cuantity, axis=1)

In [365]:
def calculate_portafolio_value(df,df_prices,  portafolio, date):
    portafolio_df = df[df['portafolio'] == f"{portafolio}"]
    prices = df_prices[df_prices['Dates'] == date]

    return portafolio_df.apply(lambda row: row['cantidad'] * prices[row['activos']], axis=1).sum()
    

print(calculate_portafolio_value(df_weights, df_prices, portafolio=1, date="2022-12-16"))
print(calculate_portafolio_value(df_weights, df_prices,  portafolio=2, date=initial_date))


304    8.952898e+08
dtype: float64
0    1.000000e+09
dtype: float64


In [366]:
def recalculate_weights(df, df_prices, date):
    prices = df_prices[df_prices['Dates'] == date]
    total_value = [calculate_portafolio_value(df, prices, portafolio, date) for portafolio in df['portafolio'].unique()]

    df['new_weights'] = df.apply(lambda x: x['cantidad']* prices[x['activos']]/total_value[int(x['portafolio'])-1], axis=1)
    
    return df

date = "2022-07-16"

df_weights = recalculate_weights(df_weights, df_prices, date="2022-07-16")
df_weights.head()

,Fecha,activos,portafolio,weight,cantidad,new_weights
0,2022-02-15,EEUU,1,0.280,2.983939e+04,0.275690
1,2022-02-15,Europa,1,0.087,1.317583e+06,0.076880
2,2022-02-15,UK,1,0.023,6.670534e+05,0.021926
3,2022-02-15,Japón,1,0.038,9.737034e+04,0.036207
4,2022-02-15,Asia Desarrollada,1,0.016,1.319250e+04,0.015294


In [367]:
def buy_sell_actives(df, df_prices, date, portafolio, active_buy, active_sell, amount):
    prices = df_prices[df_prices['Dates'] == date]

    buy_price = prices[active_buy].values[0]
    sell_price = prices[active_sell].values[0]
    buy_cuantity = amount / buy_price
    sell_cuantity = amount / sell_price
    cuantity_have_it  = df[(df['portafolio'] == f"{portafolio}") & (df['activos'] == active_sell)]['cantidad'].values[0]

    df.loc[(df['portafolio'] == f"{portafolio}") & (df['activos'] == active_buy), 'cantidad'] += buy_cuantity
    df.loc[(df['portafolio'] == f"{portafolio}") & (df['activos'] == active_sell), 'cantidad'] -= sell_cuantity

    return df


buy_sell_actives(df_weights, df_prices, portafolio=1, date="2022-07-16", active_buy="Europa", active_sell="EEUU", amount=200_000_000).head()

,Fecha,activos,portafolio,weight,cantidad,new_weights
0,2022-02-15,EEUU,1,0.280,5.337092e+03,0.275690
1,2022-02-15,Europa,1,0.087,5.197311e+06,0.076880
2,2022-02-15,UK,1,0.023,6.670534e+05,0.021926
3,2022-02-15,Japón,1,0.038,9.737034e+04,0.036207
4,2022-02-15,Asia Desarrollada,1,0.016,1.319250e+04,0.015294


In [3]:

from datetime import datetime
row = "2022-02-15 00:00:00"
date = datetime.strftime(row, '%Y-%m-%d %H:%M:%S')

2022-02-15


In [7]:
import pandas as pd
from datetime import datetime

# Sample data
data = {'Dates': [pd.Timestamp('2022-02-15 00:00:00')]}
row = data  # Assuming row is a dictionary with 'Dates' as a key

# Print the value and its type
print(row['Dates'], type(row['Dates']))

# Convert the Timestamp to a datetime object
date_time_obj = row['Dates'][0]
# Extract the date part
date = date_time_obj.date()

# Format the date as a string in the desired format (optional)
formatted_date = date.strftime('%Y-%m-%d')

print(formatted_date)  # Should print: 2022-02-15


[Timestamp('2022-02-15 00:00:00')] <class 'list'>
2022-02-15
